# quantize BGE

## example dataset

In [ ]:
# import os
# os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
# print("PYTORCH_ENABLE_MPS_FALLBACK:", os.environ.get('PYTORCH_ENABLE_MPS_FALLBACK'))

PYTORCH_ENABLE_MPS_FALLBACK: 1


In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
from tqdm import tqdm
import time
from haystack.components.generators import HuggingFaceLocalGenerator
from sklearn.metrics.pairwise import cosine_similarity
from TextEmbedder import TextEmbedder
from GetRelevantContext import GetRelevantContext
from GenerateAnswer import GenerateAnswer

In [2]:
model_name = "BAAI/bge-small-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModel.from_pretrained(model_name)

embedder = TextEmbedder(torch, tqdm, tokenizer, base_model)
get_relevant_context = GetRelevantContext(time, cosine_similarity)
generate_answer = GenerateAnswer(time, HuggingFaceLocalGenerator)

In [3]:
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")

In [4]:
embeddings = embedder.embed_dataset(dataset)

Embedding documents: 100%|██████████| 151/151 [00:08<00:00, 17.39it/s]

Embedded 151 documents.
Embedding shape: (384,)


In [5]:
prompt_template = """

You are a helpful AI assistant. You are given contexts and a question.
You must answer the question ONLY using the information given in the context.
If you do not know the answer say 'I do not know!'.

Context: {context}

Question: {question}

Answer:
"""

In [6]:
question = "What does The Colossus of Rhodes Statue look like in the context given?"

In [7]:
context = get_relevant_context.get_relevant_context(question, embeddings, embedder.embed_text, dataset)

Context retrieval time: 0.0346 seconds
'The Colossus of Rhodes (Ancient Greek: ὁ Κολοσσὸς Ῥόδιος, romanized:\xa0ho Kolossòs Rhódios Greek: Κολοσσός της Ρόδου, romanized:\xa0Kolossós tes Rhódou)[a] was a statue of the Greek sun-god Helios, erected in the city of Rhodes, on the Greek island of the same name, by Chares of Lindos in 280\xa0BC. One of the Seven Wonders of the Ancient World, it was constructed to celebrate the successful defence of Rhodes city against an attack by Demetrius Poliorcetes, who had besieged it for a year with a large army and navy.\nAccording to most contemporary descriptions, the Colossus stood approximately 70 cubits, or 33 metres (108 feet) high – approximately the height of the modern Statue of Liberty from feet to crown – making it the tallest statue in the ancient world.[2] It collapsed during the earthquake of 226 BC, although parts of it were preserved. In accordance with a certain oracle, the Rhodians did not build it again.[3] John Malalas wrote that H

In [8]:
context_text = " ".join(context[0])
answer = generate_answer.generate_answer(prompt_template, question, context_text)

Checkpoint 1: 0.0000 seconds
Checkpoint 2: 0.0282 seconds
Checkpoint 3: 3.0514 seconds
Checkpoint 4: 26.8891 seconds
Total answer generation time: 29.9687 seconds
['\nThe Colossus of Rhodes Statue is a massive bronze statue of the Greek god Helios, located in the city of Rhodes, Greece. It was built in the 3rd century BC and is one of the Seven Wonders of the Ancient World. The statue is 100 feet (30 meters) tall and weighs around 100 tons. It is considered one of the most impressive ancient statues and is still an iconic symbol of Rhodes.']


## switching to test dataset

In [3]:
test_dataset = [
    "Legal Business Name is A & R Market Inc",
    "Trade Name/Doing Business As (DBA), if applicable is Poco Loco Supermercado",
    "Address 1 is 1095 Windy Hill Rd",
    "Address 2 (apartment/suite/floor and number) is N/A",
    "City is ",
    "State is Texas",
    "Postal Code is 78640",
    "Country is United States of America",
    "Province is N/A",
    "Telephone Number is +1 (512) 312-9276",
    "Extension is N/A",
    "Federal Employer Identification Number (FEIN from IRS) is 82-4983242",
    "NAICS Code is 44511",
    "Number of current employees on payroll is 33",
    "Year Commenced Business is 2020",
    "Is the employer a closely held corporation, partnership, or sole proprietorship in which the foreign worker has an ownership interest? No",
    "Is there a familial relationship between the foreign worker and the owners, stockholders, partners, corporate officers, and/or incorporators? No"
]

In [4]:
test_dataset = [{"content": item} for item in test_dataset]

In [5]:
embeddings = embedder.embed_dataset(test_dataset)

Embedding documents: 100%|██████████| 17/17 [00:00<00:00, 43.02it/s]

Embedded 17 documents.
Embedding shape: (384,)


In [6]:
prompt_template = """

You are a helpful AI assistant. You are given context and a question.
You must answer the question ONLY using the information given in the context.
If you do not know the answer say 'I do not know!'.

Context: {context}

Question: {question}

Answer:
"""

In [7]:
question = "What is the employer zip/postal code in the context given?"

In [8]:
context = get_relevant_context.get_relevant_context(question, embeddings, embedder.embed_text, test_dataset)

Context retrieval time: 0.0298 seconds
'Postal Code is 78640'
'Federal Employer Identification Number (FEIN from IRS) is 82-4983242'
'Address 2 (apartment/suite/floor and number) is N/A'
'Telephone Number is +1 (512) 312-9276'
'Is the employer a closely held corporation, partnership, or sole proprietorship in which the foreign worker has an ownership interest? No'


In [9]:
context_text = " ".join(context)
answer = generate_answer.generate_answer(prompt_template, question, context_text)

Checkpoint 1: 0.0000 seconds
Checkpoint 2: 0.0250 seconds
Checkpoint 3: 3.0511 seconds
Checkpoint 4: 1.8197 seconds
Total answer generation time: 4.8958 seconds
["\nThe employer's zip code is 78640."]


In [7]:
question = "What is the legal business name in the context given?"

In [8]:
context = get_relevant_context.get_relevant_context(question, embeddings, embedder.embed_text, test_dataset)

Context retrieval time: 0.0258 seconds
'Legal Business Name is A & R Market Inc'
'Trade Name/Doing Business As (DBA), if applicable is Poco Loco Supermercado'
'Is the employer a closely held corporation, partnership, or sole proprietorship in which the foreign worker has an ownership interest? No'
'Is there a familial relationship between the foreign worker and the owners, stockholders, partners, corporate officers, and/or incorporators? No'
'Year Commenced Business is 2020'


In [9]:
context_text = " ".join(context)
answer = generate_answer.generate_answer(prompt_template, question, context_text)

Checkpoint 1: 0.0000 seconds
Checkpoint 2: 0.0276 seconds
Checkpoint 3: 3.0666 seconds
Checkpoint 4: 9.7323 seconds
Total answer generation time: 12.8264 seconds
['\nLegal Business Name is A & R Market Inc Trade Name/Doing Business As (DBA), if applicable is Poco Loco Supermercado Is the employer a closely held corporation, partnership, or sole proprietorship in which the foreign worker has an ownership interest? No Is there a familial relationship between the foreign worker and the owners, stockholders, partners, corporate officers, and/or incorporators? No Year Commenced Business is 2020\n\nAnswer:']


In [13]:
question = "What number of employees are on the payroll using the context given?"

In [14]:
context = get_relevant_context.get_relevant_context(question, embeddings, embedder.embed_text, test_dataset)

Context retrieval time: 0.0288 seconds
'Number of current employees on payroll is 33'
'Is the employer a closely held corporation, partnership, or sole proprietorship in which the foreign worker has an ownership interest? No'
'Federal Employer Identification Number (FEIN from IRS) is 82-4983242'
'Is there a familial relationship between the foreign worker and the owners, stockholders, partners, corporate officers, and/or incorporators? No'
'Country is United States of America'


In [15]:
context_text = " ".join(context)
answer = generate_answer.generate_answer(prompt_template, question, context_text)

Checkpoint 1: 0.0000 seconds
Checkpoint 2: 0.0003 seconds
Checkpoint 3: 7.6817 seconds
Checkpoint 4: 1.8057 seconds
Total answer generation time: 9.4877 seconds
['\nThe number of employees on the payroll is 33.']
